In [1]:
import scistanpy as ssp

In [9]:
res = ssp.results.hmc.SampleResults.from_disk(
    "amlt/hmc-pdz_trpb-small2/hmc-pdz_trpb-small2_6/trpb/libB/trpb_libB_exponential_exponential_diagnosed.nc",
    skip_fit=True,
)

In [19]:
all([])

True

In [18]:
res.inference_obj.posterior.r.values

array([[[ 3.34123e-01,  2.59330e-01,  4.24798e-01, ..., -5.63637e-02,
         -1.34181e-01, -1.06440e-02],
        [ 3.33283e-01,  3.13415e-01,  3.15025e-01, ..., -2.60759e-02,
         -1.28143e-01,  1.53810e-01],
        [ 3.22551e-01,  2.61929e-01,  2.61137e-01, ...,  6.62045e-03,
         -1.91129e-01, -1.28895e-02],
        ...,
        [ 2.77918e-01,  2.62548e-01,  3.30713e-01, ..., -6.14169e-02,
         -2.23234e-03,  2.13369e-02],
        [ 3.03370e-01,  2.99482e-01,  2.03306e-01, ..., -7.81810e-02,
         -1.01439e-01,  1.07211e-01],
        [ 2.70387e-01,  2.26512e-01,  3.52580e-01, ..., -6.23521e-02,
         -1.33274e-01, -3.85954e-02]],

       [[ 2.65400e-01,  2.71701e-01,  3.10246e-01, ..., -1.26329e-01,
         -2.55630e-01,  2.35756e-02],
        [ 3.36927e-01,  3.52690e-01,  4.16956e-01, ...,  7.54640e-02,
         -1.29539e-01,  1.23033e-01],
        [ 2.65436e-01,  2.49800e-01,  3.24871e-01, ..., -9.23061e-02,
         -3.37507e-01,  9.80250e-02],
        ...,


In [12]:
import numpy as np

In [14]:

np.array([10.0]).tolist()

[10.0]